In [1]:
import os

In [2]:
%pwd

'd:\\Chest-Cancer-Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Chest-Cancer-Classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_vgg16: Path
    updated_base_model_vgg16: Path
    base_model_vgg19: Path
    updated_base_model_vgg19: Path
    base_model_inceptionv3: Path
    updated_base_model_inceptionv3: Path
    base_model_exception: Path
    updated_base_model_exception: Path
    all_model_params: dict
    params_classes: int
    params_learning_rate: int
    updated_base_model_mobilenetv2: Path
    base_model_mobilenetv2: Path
    updated_base_model_mobilenet: Path
    base_model_mobilenet: Path
    updated_base_model_resnet: Path
    base_model_resnet: Path

In [6]:
from chestCancerClassification.constants import *
from chestCancerClassification import logger
from chestCancerClassification.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH, param_file_path=PARAMS_FILE_PATH) -> None:

        self.config = read_yaml(config_file_path)
        self.param = read_yaml(param_file_path)

        # print(self.param)

        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model

        print(self.config.prepare_base_model)

        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_vgg16=Path(config.base_model_vgg16),
            updated_base_model_vgg16=Path(config.updated_base_model_vgg16),
            base_model_vgg19=Path(config.base_model_vgg19),
            updated_base_model_vgg19=Path(config.updated_base_model_vgg19),
            base_model_resnet=Path(config.base_model_resnet),
            updated_base_model_resnet=Path(config.updated_base_model_resnet),
            base_model_mobilenet=Path(config.base_model_mobilenet),
            updated_base_model_mobilenet=Path(config.updated_base_model_mobilenet),
            base_model_mobilenetv2=Path(config.base_model_mobilenetv2),
            updated_base_model_mobilenetv2=Path(config.updated_base_model_mobilenetv2),
            base_model_inceptionv3=Path(config.base_model_inceptionv3),
            updated_base_model_inceptionv3=Path(
                config.updated_base_model_inceptionv3),
            base_model_exception=Path(config.base_model_exception),
            updated_base_model_exception=Path(
                config.updated_base_model_exception),
            all_model_params=self.param,
            params_classes= self.param.CLASSES,
            params_learning_rate = self.param.LEARNING_RATE
        )

        return prepare_base_model_config

In [8]:
import tensorflow as tf

[2023-12-15 23:41:48,653] : WARNING : module_wrapper : From d:\Chest-Cancer-Classification\env\Lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.
]


In [9]:
class Prepare_Base_Model:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    def get_base_model_vgg16(self):
        self.model = tf.keras.applications.VGG16(
            include_top=self.config.all_model_params.VGG16.INCLUDE_TOP,
            weights=self.config.all_model_params.VGG16.WEIGHTS,
            input_shape=self.config.all_model_params.VGG16.IMAGE_SIZE,
        )

    def get_base_model_vgg19(self):
        self.model = tf.keras.applications.VGG19(
            include_top=self.config.all_model_params.VGG19.INCLUDE_TOP,
            weights=self.config.all_model_params.VGG19.WEIGHTS,
            input_shape=self.config.all_model_params.VGG19.IMAGE_SIZE,
        )
    def get_base_model_resnet(self):
        self.model = tf.keras.applications.ResNet50(
            include_top=self.config.all_model_params.RESNET.INCLUDE_TOP,
            weights=self.config.all_model_params.RESNET.WEIGHTS,
            input_shape=self.config.all_model_params.RESNET.IMAGE_SIZE,
        )
    
    def get_base_model_mobilenet(self):
        self.model = tf.keras.applications.MobileNet(
            include_top=self.config.all_model_params.MOBILENET.INCLUDE_TOP,
            weights=self.config.all_model_params.MOBILENET.WEIGHTS,
            input_shape=self.config.all_model_params.MOBILENET.IMAGE_SIZE,
        )

    def get_base_model_mobilenetv2(self):
        self.model = tf.keras.applications.MobileNetV2(
            include_top=self.config.all_model_params.MOBILENETV2.INCLUDE_TOP,
            weights=self.config.all_model_params.MOBILENETV2.WEIGHTS,
            input_shape=self.config.all_model_params.MOBILENETV2.IMAGE_SIZE,
        )
    
    def get_base_model_inceptionv3(self):
        self.model = tf.keras.applications.InceptionV3(
            include_top=self.config.all_model_params.InceptionV3.INCLUDE_TOP,
            weights=self.config.all_model_params.InceptionV3.WEIGHTS,
            input_shape=self.config.all_model_params.InceptionV3.IMAGE_SIZE,
        )
    
    def get_base_model_xception(self):
        self.model = tf.keras.applications.Xception(
            include_top=self.config.all_model_params.Xception.INCLUDE_TOP,
            weights=self.config.all_model_params.Xception.WEIGHTS,
            input_shape=self.config.all_model_params.Xception.IMAGE_SIZE,
        )
    
    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model

    def update_base_model(self, save_model_path: Path):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=save_model_path,
                        model=self.full_model)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

In [10]:
config = ConfigurationManager()
prepare_base_model_config = config.get_prepare_base_model_config()
# print(prepare_base_model_config)

[2023-12-15 23:41:49,154] : INFO : common : yaml file: config\config.yaml loaded successfully]
[2023-12-15 23:41:49,159] : INFO : common : yaml file: params.yaml loaded successfully]
[2023-12-15 23:41:49,160] : INFO : common : created directory at: artifacts]
{'root_dir': 'artifacts/base_models', 'base_model_vgg16': 'artifacts/base_models/vgg16.h5', 'updated_base_model_vgg16': 'artifacts/base_models/updated_vgg16.h5', 'base_model_vgg19': 'artifacts/base_models/vgg19.h5', 'updated_base_model_vgg19': 'artifacts/base_models/updated_vgg19.h5', 'base_model_inceptionv3': 'artifacts/base_models/inceptionv3.h5', 'updated_base_model_inceptionv3': 'artifacts/base_models/updated_inceptionv3.h5', 'base_model_exception': 'artifacts/base_models/exception.h5', 'updated_base_model_exception': 'artifacts/base_models/updated_exception.h5', 'base_model_resnet': 'artifacts/base_models/resnet.h5', 'updated_base_model_resnet': 'artifacts/base_models/updated_resnet.h5', 'base_model_mobilenet': 'artifacts/bas

In [11]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = Prepare_Base_Model(config=prepare_base_model_config)
    
    logger.info("Preparing base model VGG16")
    prepare_base_model.get_base_model_vgg16()
    prepare_base_model.update_base_model(prepare_base_model_config.updated_base_model_vgg16)
    
    logger.info("base model VGG16 updated successfully")
    
    logger.info("Preparing base model VGG19")
    prepare_base_model.get_base_model_vgg19()
    prepare_base_model.update_base_model(prepare_base_model_config.updated_base_model_vgg19)
    
    logger.info("base model VGG19 updated successfully")
    
    logger.info("Preparing base model RESNET")
    prepare_base_model.get_base_model_resnet()
    prepare_base_model.update_base_model(
    prepare_base_model_config.updated_base_model_resnet)

    logger.info("base model RESNET updated successfully")

    logger.info("Preparing base model Xception")
    prepare_base_model.get_base_model_xception()
    prepare_base_model.update_base_model(
    prepare_base_model_config.updated_base_model_exception)

    logger.info("base model Xception updated successfully")

    logger.info("Preparing base model inceptionv3")
    prepare_base_model.get_base_model_inceptionv3()
    prepare_base_model.update_base_model(
    prepare_base_model_config.updated_base_model_inceptionv3)

    logger.info("base model inceptionv3 updated successfully")

    logger.info("Preparing base model Mobilenet")
    prepare_base_model.get_base_model_mobilenet()
    prepare_base_model.update_base_model(
    prepare_base_model_config.updated_base_model_mobilenet)

    logger.info("base model Mobilenet updated successfully")

    logger.info("Preparing base model mobilenetv2")
    prepare_base_model.get_base_model_mobilenetv2()
    prepare_base_model.update_base_model(
    prepare_base_model_config.updated_base_model_mobilenetv2)

    logger.info("base model mobilenetv2 updated successfully")
    
except Exception as e:
    raise e

[2023-12-15 23:41:49,173] : INFO : common : yaml file: config\config.yaml loaded successfully]
[2023-12-15 23:41:49,178] : INFO : common : yaml file: params.yaml loaded successfully]
[2023-12-15 23:41:49,179] : INFO : common : created directory at: artifacts]
{'root_dir': 'artifacts/base_models', 'base_model_vgg16': 'artifacts/base_models/vgg16.h5', 'updated_base_model_vgg16': 'artifacts/base_models/updated_vgg16.h5', 'base_model_vgg19': 'artifacts/base_models/vgg19.h5', 'updated_base_model_vgg19': 'artifacts/base_models/updated_vgg19.h5', 'base_model_inceptionv3': 'artifacts/base_models/inceptionv3.h5', 'updated_base_model_inceptionv3': 'artifacts/base_models/updated_inceptionv3.h5', 'base_model_exception': 'artifacts/base_models/exception.h5', 'updated_base_model_exception': 'artifacts/base_models/updated_exception.h5', 'base_model_resnet': 'artifacts/base_models/resnet.h5', 'updated_base_model_resnet': 'artifacts/base_models/updated_resnet.h5', 'base_model_mobilenet': 'artifacts/bas

d:\Chest-Cancer-Classification\env\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

AttributeError: 'PrepareBaseModelConfig' object has no attribute 'updated_base_model_resnet'